<a href="https://colab.research.google.com/github/nathanbollig/distributed-mutation/blob/main/explore_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explore Synthetic Data and Classifier

In Phase 1 and 2 of this project, synthetic viral sequence data is generated and a classification model is trained on 80% of this data. The result of running the `phases_1_and_2.py` script is a tuple of results objects saved to disk in a pickle file. After performing Phase 1 and Phase 2, this notebook loads in this results file and explores its contents.

In [1]:
import numpy as np

## Phase 1 and 2

This section follows the instructions in the project README to generate data and train a classifier.

In [2]:
# Clone the git repository
!git clone https://github.com/nathanbollig/distributed-mutation

Cloning into 'distributed-mutation'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 22 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [3]:
# Apply requirements.txt
%cd distributed-mutation
!pip install -r requirements.txt

/content/distributed-mutation
     |████████████████████████████████| 374 kB 5.1 MB/s 
     |████████████████████████████████| 312 kB 41.6 MB/s 
     |████████████████████████████████| 7.1 MB 15.2 MB/s 
     |████████████████████████████████| 110.5 MB 1.2 kB/s 
     |████████████████████████████████| 50 kB 5.4 MB/s 
     |████████████████████████████████| 503 kB 54.6 MB/s 
     |████████████████████████████████| 3.8 MB 44.8 MB/s 
     |████████████████████████████████| 2.9 MB 43.0 MB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=969e5c2adad5ef5d9a6f582a7f3c1a56c29807ce1fecdd6938eeeab75fdfd932
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall:

The above conflicts are acceptable because these conflicts are with pre-installed packages in this environment that I don't believe are needed for executing my code. May now need to restart the runtime (and therefore change directory again).

In [4]:
# Run script
%cd distributed-mutation
!bash phases_1_and_2.sh

[Errno 2] No such file or directory: 'distributed-mutation'
/content/distributed-mutation
Using TensorFlow backend.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10


2021-11-11 20:55:43.333221: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-11 20:55:43.336522: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-11-11 20:55:43.336911: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55abceb66a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-11 20:55:43.336947: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version



8000/8000 [==============================] - 0s 36us/step - loss: 0.6137 - acc: 0.7147
Epoch 2/10
8000/8000 [==============================] - 0s 19us/step

## Explore Phase 1 and 2 Output

### Read in results

In [5]:
import pickle

In [6]:
with open("phase_1_2_results.pkl", 'rb') as pfile:
    big_bang_result_tuple = pickle.load(pfile)

Using TensorFlow backend.












Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




### Unpack items

In [7]:
model, result, X_list, y_list, gen, aa_vocab = big_bang_result_tuple

In [8]:
X_train, X_val, X_test = X_list
y_train, y_val, y_test = y_list

### Model and validation results

The Keras model object is stored in `model`.

In [9]:
type(model)

keras.engine.sequential.Sequential

During the training of this model, a dictionary was created of training set and validation set performance. We can display the values in this dictionary to see the accuracy of the stored model on its training and validation set.

In [10]:
result

{'model_train_accuracy': 0.98075, 'model_val_accuracy': 0.965}

### Synthetic sequence data

The data are sequences (variable names prefixed with `X`) and their labels (prefixed with `y`). Data is split into training, validation, and test sets. The training set was used to train the model and the validation set was used to measure the performance reported in the `result` dictionary. The test set has not yet been used for model training or evaluation.

The data should be 80% training, 10% validation, and 10% test, with the total number of sequences as specified in the `phases_1_and_2.sh` script.

In [11]:
len(X_train), len(X_val), len(X_test)

(8000, 1000, 1000)

As expected, there are the same number of labels.

In [12]:
len(y_train), len(y_val), len(y_test)

(8000, 1000, 1000)

The sequence variables (`X_`*) are numpy arrays, where each sequence is represented by a 60 x 20 matrix.

In [13]:
X_train.shape

(8000, 60, 20)

Each of the 60 positions in the sequence is represented by a one-hot vector of length 20. We assume that 20 is the size of the character alphabet. For example, a single sequence looks like the following matrix.

In [14]:
X_train[42]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

The binary sequence label is an integer. The value 1 represents positive, and 0 represents negative. For example, the label for the above sequence is shown below.

In [15]:
y_train[42]

1

### Applying the model to a sequence

We can apply the stored model to a sequence to get a prediction, using the TensorFlow model objects's API. For example, suppose we want to apply the model to sequences at index 42 and 43 in the training set.

In [16]:
model.predict(X_train[42:44])

array([[0.46583968],
       [0.92461646]], dtype=float32)

We see the prediction for each sequence as a number between 0 and 1. In this case, they are both close to 1, indicating higher confidence of positive. These predictions are correct in this context, as shown below.

In [17]:
y_train[42:44]

array([1, 1])

### Amino acid vocabulary

The pickle file also included the amino acid vocabulary used for the encoding.

In [18]:
print(aa_vocab)

['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']


In [19]:
len(aa_vocab)

20

This associates an index in the range 0-19 (as described above in relation to the one-hot representation of sequences) to a specific character that reflects an amino acid in the biological sequence.

### Markov model (HMMGenerator object)

Finally, the pickle file includes the `HMMGenerator` object used to synthesize the data.

In [20]:
type(gen)

HMM_generator_motif.HMMGenerator

This object has fields that determine the structure behind the synthesized data. For example, the sequence lengths, where the active site starts in a sequence, how long the active site is, the class proportion, the intensity of the positive class signal (as described in my report), emission probability distributions, transition mutation probabilities, and some others.

In [21]:
print(gen.__dict__.keys())

dict_keys(['seq_length', 'start', 'active_site_length', 'p', 'class_signal', 'aa_list', 'background_emission', 'state0_emission', 'state1_emissions', 'transmat', 'startprob', 'emissionprob', 'n_components', 'model'])


The `HMMGenerator` class is a wrapper around a Multinomial Hidden Markov Model implemented in `hmmlearn`. The field `gen.model` contains the `hmmlearn` model used to synthesize data.

In [22]:
type(gen.model)

hmmlearn.hmm.MultinomialHMM

## Using the generator to classify novel sequences

The `HMMGenerator` class is capable of predicting the class 1 probability of a sequence under the existing HMM. Suppose we take a test item.

In [23]:
x = X_test[42]

In [24]:
y = y_test[42]
y

1

This is a negative instance. Make a mutation at position 35.

In [25]:
x

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [26]:
x[25]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [27]:
x[25][16]

0.0

In [28]:
x[25][16] = 0.0

In [29]:
x[25][11] = 1.0

In [30]:
aa_vocab[16]

'T'

In [31]:
aa_vocab[11]

'K'

This corresponds to substituting the 16th character in the alphabet ('T') with the 11th character ('K'). Now we can predict the class label using the model (after reshaping into a batch of size 1), and predict the class label under the generator model (after converted to a sequence of indices rather than one-hot encoding).

In [32]:
# Model prediction on mutation
model.predict(x.reshape(1,60,20))

array([[0.6896637]], dtype=float32)

In [33]:
# Generator posterior prob of positive class
gen.predict_proba(np.argmax(x, axis=1))

0.9979396729029935

Make a few more mutations that I know should make it look like a positive sequence, then see that the generator indicates this looks more like a positive sequence, and the model correctly predicts this as well.

In [34]:
import numpy as np

# Set the following characters starting at index 26: 'RSFIED'
chars = 'RSFIED'

for i in range(26, 32):
    # Get the new char index
    char = chars[i-26]
    new_char_idx = aa_vocab.index(char)

    # Reset current char to 0
    x[i][np.argmax(x[i])] = 0.0

    # Make substitution for new char
    x[i][new_char_idx] = 1.0

In [35]:
# Generator posterior prob of positive class
gen.predict_proba(np.argmax(x, axis=1))

0.9999998716679468

In [36]:
# Model prediction on mutation
model.predict(x.reshape(1,60,20))

array([[0.9757017]], dtype=float32)